# 1.5 詹森阿尔法 (Jensen's Alpha)

## 概念介绍

詹森阿尔法(Jensen's Alpha)，简称阿尔法，由迈克尔·詹森(Michael Jensen)于1968年提出，是一种风险调整后的绩效评估指标。它衡量了投资组合相对于资本资产定价模型(CAPM)预测的超额收益，即超出市场给予的风险溢价的部分。

### 詹森阿尔法公式

$\alpha_J = R_p - [R_f + \beta_p(R_m - R_f)]$

其中：
- $\alpha_J$ 是詹森阿尔法
- $R_p$ 是投资组合的实际收益率
- $R_f$ 是无风险收益率
- $\beta_p$ 是投资组合的Beta系数
- $R_m$ 是市场收益率
- $R_f + \beta_p(R_m - R_f)$ 是根据CAPM模型计算的预期收益率

正的阿尔法值表示投资组合表现超过其风险水平所预期的收益，即投资经理产生了超额收益。负的阿尔法值表示投资组合表现不及预期。Alpha = 0表示投资组合正好获得了与其风险水平相匹配的收益。

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib statsmodels seaborn

## 1.2 从Github加载数据

In [ ]:
!wget -O AAPL_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/AAPL_data.csv"
!wget -O SPY_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/SPY_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_spy = "SPY_data.csv"

## 1.3 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 假设的无风险利率
risk_free_rate = 0.02 / 252  # 年利率2%转换为日利率

# 读取数据
try:
    # 读取 CSV 数据
    column_names = ["Date", "Open", "High", "Low", "Close", "Volume"]
    
    data_aapl = pd.read_csv(file_path_aapl, skiprows=3, names=column_names)
    data_spy = pd.read_csv(file_path_spy, skiprows=3, names=column_names)
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")
    # 如果加载失败，尝试不跳过行读取
    try:
        data_aapl = pd.read_csv(file_path_aapl)
        data_spy = pd.read_csv(file_path_spy)
        print("重新尝试数据加载成功")
    except Exception as e:
        print(f"数据加载再次失败: {e}")

In [ ]:
# 数据预览
print("AAPL数据预览:")
display(data_aapl.head())
print("\nSPY数据预览:")
display(data_spy.head())

In [ ]:
# 统一列名
data_aapl.rename(columns={'Close': 'Adj Close'}, inplace=True)
data_spy.rename(columns={'Close': 'Adj Close'}, inplace=True)

# 转换日期格式
try:
    data_aapl['Date'] = pd.to_datetime(data_aapl['Date'], format="%Y/%m/%d")
except Exception as e:
    print(f"AAPL日期转换错误，尝试自动检测格式: {e}")
    data_aapl['Date'] = pd.to_datetime(data_aapl['Date'])

try:    
    data_spy['Date'] = pd.to_datetime(data_spy['Date'], format="%Y/%m/%d")
except Exception as e:
    print(f"SPY日期转换错误，尝试自动检测格式: {e}")
    data_spy['Date'] = pd.to_datetime(data_spy['Date'])

# 设置索引
data_aapl.set_index('Date', inplace=True)
data_spy.set_index('Date', inplace=True)

In [ ]:
# 计算每日收益率
data_aapl['Daily Return'] = data_aapl['Adj Close'].pct_change()
data_spy['Daily Return'] = data_spy['Adj Close'].pct_change()

# 清除缺失值
data_aapl = data_aapl.dropna()
data_spy = data_spy.dropna()

# 数据基本统计量
print("AAPL日收益率统计:")
display(data_aapl['Daily Return'].describe())
print("\nSPY日收益率统计:")
display(data_spy['Daily Return'].describe())

## 1.5 对齐数据集并计算Beta系数

In [ ]:
# 确保两个DataFrame具有相同的索引
# 寻找两个数据集的共同日期
common_dates = data_aapl.index.intersection(data_spy.index)
data_aapl_aligned = data_aapl.loc[common_dates]
data_spy_aligned = data_spy.loc[common_dates]

# 验证对齐后的数据大小
print(f"AAPL对齐后数据量: {len(data_aapl_aligned)}")
print(f"SPY对齐后数据量: {len(data_spy_aligned)}")

In [ ]:
# 计算AAPL的Beta值
X = sm.add_constant(data_spy_aligned['Daily Return'])
y = data_aapl_aligned['Daily Return']

model = sm.OLS(y, X).fit()
beta_aapl = model.params.iloc[1]
alpha_regression = model.params.iloc[0]  # 回归截距

# 显示回归结果
print(model.summary())
print(f"\nAAPL Beta: {beta_aapl:.4f}")
print(f"回归截距(Alpha): {alpha_regression:.6f}")

## 1.6 计算詹森阿尔法

In [ ]:
# 计算 AAPL 和 SPY 的平均回报
mu_aapl = data_aapl_aligned['Daily Return'].mean()
mu_spy = data_spy_aligned['Daily Return'].mean()

# 根据CAPM模型计算AAPL的预期收益率
expected_return_aapl = risk_free_rate + beta_aapl * (mu_spy - risk_free_rate)

# 计算 Jensen's Alpha（日度）
jensens_alpha_aapl = mu_aapl - expected_return_aapl

# 计算年化值
annual_mu_aapl = mu_aapl * 252
annual_mu_spy = mu_spy * 252
annual_risk_free = risk_free_rate * 252
annual_expected_return = annual_risk_free + beta_aapl * (annual_mu_spy - annual_risk_free)
annual_jensens_alpha = annual_mu_aapl - annual_expected_return

# 打印结果
print("日度计算结果:")
print(f"AAPL的实际平均收益率: {mu_aapl:.6f}")
print(f"SPY的平均收益率: {mu_spy:.6f}")
print(f"CAPM预期收益率: {expected_return_aapl:.6f}")
print(f"AAPL的詹森阿尔法: {jensens_alpha_aapl:.6f}")

print("\n年化计算结果:")
print(f"AAPL的年化平均收益率: {annual_mu_aapl:.4f} ({annual_mu_aapl:.2%})")
print(f"SPY的年化平均收益率: {annual_mu_spy:.4f} ({annual_mu_spy:.2%})")
print(f"年化无风险收益率: {annual_risk_free:.4f} ({annual_risk_free:.2%})")
print(f"年化CAPM预期收益率: {annual_expected_return:.4f} ({annual_expected_return:.2%})")
print(f"AAPL的年化詹森阿尔法: {annual_jensens_alpha:.4f} ({annual_jensens_alpha:.2%})")

## 1.7 结果可视化

In [ ]:
# 绘制证券市场线(SML)和AAPL位置
plt.figure(figsize=(12, 8))

# 绘制证券市场线
beta_range = np.linspace(0, 2, 100)
expected_returns = annual_risk_free + beta_range * (annual_mu_spy - annual_risk_free)
plt.plot(beta_range, expected_returns, 'r-', linewidth=2, label='证券市场线(SML)')

# 标记重要点
plt.scatter(0, annual_risk_free, s=100, color='green', label='无风险资产', zorder=5)
plt.scatter(1, annual_mu_spy, s=100, color='blue', label='市场组合(SPY)', zorder=5)
plt.scatter(beta_aapl, annual_mu_aapl, s=150, color='red', label='AAPL(实际)', zorder=5)
plt.scatter(beta_aapl, annual_expected_return, s=100, color='purple', marker='x', label='AAPL(预期)', zorder=5)

# 绘制詹森阿尔法
plt.plot([beta_aapl, beta_aapl], [annual_expected_return, annual_mu_aapl], 'g--', linewidth=2, label='詹森阿尔法')

# 设置图表属性
plt.title('证券市场线(SML)与詹森阿尔法', fontsize=16)
plt.xlabel('Beta系数', fontsize=12)
plt.ylabel('年化预期收益率', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 添加注释
plt.annotate(f"Jensen's Alpha: {annual_jensens_alpha:.2%}", 
             xy=(beta_aapl, (annual_mu_aapl + annual_expected_return)/2),
             xytext=(beta_aapl + 0.2, (annual_mu_aapl + annual_expected_return)/2),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# 计算实际收益率与CAPM预期收益率的差异（日度）
# 为每个日期计算CAPM预期收益
data_spy_aligned['Expected Return AAPL'] = risk_free_rate + beta_aapl * (data_spy_aligned['Daily Return'] - risk_free_rate)

# 计算每日Alpha（实际收益 - 预期收益）
data_aapl_aligned['Daily Alpha'] = data_aapl_aligned['Daily Return'] - data_spy_aligned['Expected Return AAPL']

# 绘制每日Alpha的时间序列
plt.figure(figsize=(12, 6))
plt.plot(data_aapl_aligned.index, data_aapl_aligned['Daily Alpha'], 'b-', alpha=0.6)
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.axhline(y=jensens_alpha_aapl, color='g', linestyle='--', linewidth=2, label=f'平均Alpha: {jensens_alpha_aapl:.6f}')
plt.fill_between(data_aapl_aligned.index, data_aapl_aligned['Daily Alpha'], 0, 
                 where=(data_aapl_aligned['Daily Alpha'] >= 0), color='g', alpha=0.3, label='正Alpha')
plt.fill_between(data_aapl_aligned.index, data_aapl_aligned['Daily Alpha'], 0, 
                 where=(data_aapl_aligned['Daily Alpha'] < 0), color='r', alpha=0.3, label='负Alpha')

plt.title('AAPL的每日詹森阿尔法', fontsize=16)
plt.xlabel('日期', fontsize=12)
plt.ylabel('每日Alpha值', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 绘制CAPM拟合图
plt.figure(figsize=(12, 8))

# 散点图：市场收益率vs股票收益率
plt.scatter(data_spy_aligned['Daily Return'], data_aapl_aligned['Daily Return'], 
            alpha=0.5, label='日收益率数据点')

# 回归线
x_range = np.linspace(data_spy_aligned['Daily Return'].min(), data_spy_aligned['Daily Return'].max(), 100)
y_pred = model.params[0] + model.params[1] * x_range
plt.plot(x_range, y_pred, 'r', linewidth=2, label=f'回归线 (Beta = {beta_aapl:.4f})')

# 标记Alpha
plt.plot([0], [model.params[0]], 'go', markersize=10)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
plt.annotate(f"Alpha = {alpha_regression:.6f}", 
             xy=(0, model.params[0]), 
             xytext=(0.001, model.params[0] + 0.001),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12)

# 设置图表属性
plt.title('CAPM拟合与詹森阿尔法', fontsize=16)
plt.xlabel('市场(SPY)日收益率', fontsize=12)
plt.ylabel('AAPL日收益率', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 添加注释
plt.annotate(f"Beta = {beta_aapl:.4f}\nR² = {model.rsquared:.4f}\nJensen's Alpha = {jensens_alpha_aapl:.6f}", 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             fontsize=12,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

## 1.8 结果分析与解释

通过上述计算和可视化，我们可以得出以下结论：

1. **詹森阿尔法解读**：
   - AAPL的詹森阿尔法为{annual_jensens_alpha:.2%}（年化）
   - {'正' if annual_jensens_alpha > 0 else '负'}的阿尔法值表示AAPL的实际表现{'超过' if annual_jensens_alpha > 0 else '低于'}了根据CAPM模型预期的收益
   - 这说明苹果公司在考虑其系统性风险后，{'产生了超额收益' if annual_jensens_alpha > 0 else '表现不及预期'}

2. **CAPM模型分析**：
   - AAPL的Beta系数为{beta_aapl:.4f}，表明其{'波动性大于市场' if beta_aapl > 1 else '波动性小于市场'}
   - 根据CAPM模型，AAPL的预期年化收益率应为{annual_expected_return:.2%}
   - 实际年化收益率为{annual_mu_aapl:.2%}，与预期{'存在显著差异' if abs(annual_mu_aapl - annual_expected_return) > 0.01 else '基本一致'}

3. **时间序列分析**：
   - 从每日Alpha的时间序列图可以看出，AAPL在大部分时间内的表现{'高于' if np.sum(data_aapl_aligned['Daily Alpha'] > 0) > len(data_aapl_aligned)/2 else '低于'}CAPM预期
   - Alpha值的{'波动性较大' if data_aapl_aligned['Daily Alpha'].std() > 0.01 else '波动性较小'}，表明公司绩效相对于市场预期存在{'显著' if data_aapl_aligned['Daily Alpha'].std() > 0.01 else '轻微'}波动

4. **回归分析结果**：
   - 回归截距（Alpha）为{alpha_regression:.6f}，这与我们计算的詹森阿尔法{'接近' if abs(alpha_regression - jensens_alpha_aapl) < 0.0001 else '有所不同'}
   - R²值为{model.rsquared:.4f}，表明市场收益率能解释AAPL收益率变动的{model.rsquared:.2%}
   - 这意味着AAPL的收益有{(1-model.rsquared):.2%}来自非系统性因素

5. **投资策略建议**：
   - 詹森阿尔法是评估投资组合管理者创造超额收益能力的重要指标
   - {'正' if annual_jensens_alpha > 0 else '负'}的阿尔法值表明投资AAPL{'可能是有利的' if annual_jensens_alpha > 0 else '需要谨慎考虑'}
   - 投资者应注意，历史表现不代表未来结果，阿尔法可能随时间变化
   - 同时考虑詹森阿尔法、夏普比率和特雷诺比率可以获得更全面的风险调整收益评估

6. **阿尔法来源分析**：
   - 阿尔法可能来自公司特有的因素，如管理层决策、创新能力、品牌价值等
   - 也可能来自市场异常或CAPM模型假设的不完善